In [ ]:
from PIL import Image, ImageFilter
import numpy as np
from scipy.ndimage import label


Histogram equalization

In [ ]:
# dataset\coins_images\coins_images\euro_coins\0bececd88b.jpg
# Step 1: Load the input image
image_path = "../coin-counting/dataset/coins_images/coins_images/euro_coins/0bececd88b.jpg" 
image = Image.open(image_path).convert("L").filter(ImageFilter.MinFilter(size=3)).filter(ImageFilter.CONTOUR)  # Convert to grayscale

# Step 2: Perform histogram equalization
pixel_array = np.array(image)  # Convert image to NumPy array
histogram, bin_edges = np.histogram(pixel_array, bins=256, range=(0, 255))

# Cumulative Distribution Function (CDF)
cdf = histogram.cumsum()
cdf_normalized = cdf * (255 / cdf[-1])  # Normalize to scale 0-255

# Map the original pixel values to equalized values
equalized_array = np.interp(pixel_array.flatten(), bin_edges[:-1], cdf_normalized)
equalized_array = equalized_array.reshape(pixel_array.shape).astype("uint8")

# Convert back to Pillow Image and save/display
equalized_image = Image.fromarray(equalized_array)

Image.open(image_path).show()
# equalized_image.save("contrast_equalized_image.jpg")
equalized_image.show()

Contrast stretching

In [ ]:
# Step 1: Load the input image
image_path = "../coin-counting/dataset/coins_images/coins_images/euro_coins/0bececd88b.jpg"
image = Image.open(image_path).convert("L")  # Convert to grayscale

# Step 2: Noise Reduction (Median Filter)
filtered_image = image.filter(ImageFilter.MedianFilter(size=3))

# Step 3: Perform Histogram Equalization
pixel_array = np.array(filtered_image)
histogram, bin_edges = np.histogram(pixel_array, bins=256, range=(0, 255))

cdf = histogram.cumsum()
cdf_normalized = cdf * (255 / cdf[-1])

equalized_array = np.interp(pixel_array.flatten(), bin_edges[:-1], cdf_normalized)
equalized_array = equalized_array.reshape(pixel_array.shape).astype("uint8")

# Step 4: Thresholding (Binarization)
binary_array = equalized_array > np.mean(equalized_array)  # Simple global thresholding
binary_image = Image.fromarray((binary_array * 255).astype("uint8"))  # Convert to binary image (0 or 255)

# Step 5: Morphological Operations (Optional)
# Fill holes and clean up small noise (example using scipy.ndimage)
labeled_array, num_features = label(binary_array)  # Label connected components

# Step 6: Display and Save Results
binary_image.save("processed_binary_image.jpg")
binary_image.show()

print(f"Number of detected coins (connected components): {num_features}")

In [ ]:
# Step 1: Load the input image
image_path = "../coin-counting/dataset/coins_images/coins_images/all_coins/1a1db517f0.jpg"  # Replace with your image file path
image = Image.open(image_path).convert("L")  # Convert to grayscale

# Step 2: Perform contrast stretching
pixel_array = np.array(image)  # Convert image to NumPy array
min_val, max_val = pixel_array.min(), pixel_array.max()  # Find min and max pixel values

# Stretch the pixel values to 0-255
stretched_array = ((pixel_array - min_val) * (255 / (max_val - min_val))).astype("uint8")

# Convert back to Pillow Image and save/display
stretched_image = Image.fromarray(stretched_array)
stretched_image.save("contrast_stretched_image.jpg")
stretched_image.show()


In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from scipy.ndimage import binary_fill_holes, label
from skimage.filters import threshold_otsu
from skimage.morphology import dilation, erosion, disk
from matplotlib import pyplot as plt

def preprocess_image(image_path):
    """Load and preprocess the image."""
    # Load the image and convert to grayscale
    image = Image.open(image_path).convert("L")
    # Convert to NumPy array and apply histogram equalization
    image_array = np.array(image)
    image_equalized = np.clip((image_array - image_array.min()) * 
                              (255 / (image_array.max() - image_array.min())), 0, 255).astype("uint8")
    return image_array, image_equalized

def segment_image(image):
    """Segment the image using Otsu's thresholding."""
    # Apply Otsu's thresholding
    thresh = threshold_otsu(image)
    binary_mask = image > thresh
    # Fill holes in the binary mask
    filled_mask = binary_fill_holes(binary_mask)
    return filled_mask

def clean_mask(binary_mask):
    """Apply morphological operations to clean the binary mask."""
    # Use closing operation: dilation followed by erosion
    structuring_element = disk(5)
    cleaned_mask = dilation(binary_mask, structuring_element)
    cleaned_mask = erosion(cleaned_mask, structuring_element)
    return cleaned_mask

def count_objects(cleaned_mask):
    """Count objects in the binary mask using connected component analysis."""
    labeled_array, num_features = label(cleaned_mask)
    return labeled_array, num_features

def process_dataset(dataset_dir, csv_path):
    """Process the dataset and compute predictions."""
    predictions = {}
    # Load the ground truth CSV
    ground_truth = pd.read_csv(csv_path)
    # Iterate through images in the dataset
    for _, row in ground_truth.iterrows():
        image_name = row['image_name']
        image_path = os.path.join(dataset_dir, image_name)
        # Process the image
        try:
            _, preprocessed_image = preprocess_image(image_path)
            binary_mask = segment_image(preprocessed_image)
            cleaned_mask = clean_mask(binary_mask)
            _, num_features = count_objects(cleaned_mask)
            # Store the prediction
            predictions[image_name] = num_features
        except Exception as e:
            predictions[image_name] = None  # Handle errors gracefully
    return predictions

def calculate_error_fixed(csv_path, predictions):
    """Calculate error percentage for predictions compared to the CSV, with error handling."""
    ground_truth = pd.read_csv(csv_path)
    ground_truth['predicted_count'] = ground_truth['image_name'].map(predictions)
    # Handle cases where predictions are missing or invalid
    ground_truth['absolute_error'] = abs(
        ground_truth['coins_count'] - ground_truth['predicted_count']
    ).fillna(0).astype(float)
    # Calculate percentage error and handle division by zero or missing values
    ground_truth['percentage_error'] = (
        (ground_truth['absolute_error'] / ground_truth['coins_count'].replace(0, np.nan)) * 100
    ).fillna(0).round(2)
    # Calculate average error percentage
    average_error_percentage = ground_truth['percentage_error'].mean().round(2)
    return ground_truth, average_error_percentage

def visualize_results(original_image, labeled_array, num_features):
    """Visualize the results."""
    plt.figure(figsize=(10, 10))
    # Display the original image with labeled regions
    plt.imshow(original_image, cmap="gray")
    plt.title(f"Detected Coins: {num_features}")
    plt.axis("off")
    plt.show()

# Define paths
dataset_dir = "../coin-counting/dataset/coins_images/coins_images/all_coins"  # Replace with the actual folder path for images
csv_path = "../coin-counting/coins_count_values.csv"

# Process the dataset
predictions = process_dataset(dataset_dir, csv_path)

# Perform error analysis
error_analysis_fixed, avg_error_percentage_fixed = calculate_error_fixed(csv_path, predictions)

# Display the error analysis and average error percentage
print("Average Error Percentage:", avg_error_percentage_fixed)
error_analysis_fixed.to_csv("error_analysis_results.csv", index=False)


In [2]:
from PIL import Image, ImageDraw
import numpy as np
from scipy.ndimage import binary_fill_holes, label
from scipy.ndimage import gaussian_filter
from skimage.filters import threshold_otsu
from skimage.measure import regionprops
import matplotlib.pyplot as plt
import os


def preprocess_image(image_path):
    """Preprocess the image: Convert to grayscale and apply Gaussian blur."""
    # Load image
    image = Image.open(image_path).convert("L")  # Convert to grayscale
    
    # Convert to NumPy array
    image_array = np.array(image)
    
    # Apply Gaussian blur
    blurred_image = gaussian_filter(image_array, sigma=3)
    
    return image_array, blurred_image


def segment_image(blurred_image):
    """Segment the image using Otsu's thresholding."""
    # Apply Otsu's thresholding
    thresh = threshold_otsu(blurred_image)
    binary_mask = blurred_image > thresh
    
    # Fill holes in the binary mask
    filled_mask = binary_fill_holes(binary_mask)
    
    return filled_mask


def clean_mask(binary_mask):
    """Clean the binary mask using basic morphological operations."""
    # Remove small noise
    cleaned_mask = binary_fill_holes(binary_mask)
    return cleaned_mask


def detect_and_count_coins(cleaned_mask, original_image):
    """Detect and count coins using connected components."""
    # Label connected components
    labeled_array, num_features = label(cleaned_mask)
    
    # Measure properties of labeled regions
    regions = regionprops(labeled_array)
    
    # Prepare output image with marked coins
    output_image = Image.fromarray(original_image).convert("RGB")
    draw = ImageDraw.Draw(output_image)
    
    # Draw circles and annotate
    for region in regions:
        # Get centroid and equivalent diameter
        centroid = region.centroid
        diameter = region.equivalent_diameter
        x, y = int(centroid[1]), int(centroid[0])
        radius = int(diameter / 2)
        
        # Draw circle
        draw.ellipse((x - radius, y - radius, x + radius, y + radius), outline="green", width=3)
        # Annotate
        draw.text((x, y), f"{radius:.1f}", fill="yellow")
    
    return output_image, num_features


def process_images_in_folder(folder_path):
    """Process all images in a folder and count coins."""
    results = {}
    
    # Iterate through all images in the folder
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, file_name)
            print(f"Processing {file_name}...")
            
            # Preprocess the image
            original_image, blurred_image = preprocess_image(image_path)
            
            # Segment the image
            binary_mask = segment_image(blurred_image)
            
            # Clean the binary mask
            cleaned_mask = clean_mask(binary_mask)
            
            # Detect and count coins
            output_image, coin_count = detect_and_count_coins(cleaned_mask, original_image)
            
            # Save results
            results[file_name] = coin_count
            
            # Show processed image
            plt.figure(figsize=(10, 10))
            plt.imshow(output_image)
            plt.title(f"Detected Coins: {coin_count}")
            plt.axis("off")
            plt.show()
    
    return results


# Main Execution
if __name__ == "__main__":
    # Path to the folder containing coin images
    folder_path = "../coin-counting/dataset/coins_images/coins_images/euro_coins"  # Update the path
    
    # Process images and count coins
    coin_counts = process_images_in_folder(folder_path)
    
    # Print results
    print("\nFinal Results:")
    for image_name, count in coin_counts.items():
        print(f"{image_name}: {count} coins")


ModuleNotFoundError: No module named 'skimage'

In [1]:
import logging
from Classes.ImagePreprocessor import ImageProcessor

def main():
    # Configuration du logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    # Chemin vers l'image de test
    image_path = '../coin-counting/dataset/coins_images/coins_images/all_coins/0c0f693a3e.jpg'  # Remplacez par le chemin vers votre image de test
    output_path = '../coin-counting'  # Remplacez par le chemin vers le fichier de sortie
    
    # Initialiser l'instance de ImageProcessor
    try:
        processeur = ImageProcessor(image_path)
    except Exception as e:
        logging.error(f"Erreur lors de l'initialisation de ImageProcessor: {e}")
        return

    # Afficher l'image originale
    logging.info("Affichage de l'image originale")
    processeur.show_image()

    # Traiter l'image de manière dynamique
    try:
        image_traitee = processeur.traiter_image()
        image_traitee.show()
        logging.info("Image traitée dynamiquement avec succès")
    except Exception as e:
        logging.error(f"Erreur lors du traitement dynamique de l'image : {e}")
        return
    
    # Sauvegarder l'image traitée
    try:
        processeur.save_image(output_path, image=image_traitee)
    except Exception as e:
        logging.error(f"Erreur lors de la sauvegarde de l'image : {e}")
        return

    # Afficher les images avant et après traitement
    try:
        processeur.afficher_images_avant_apres()
    except Exception as e:
        logging.error(f"Erreur lors de l'affichage des images avant et après : {e}")
        return

    # Test avec le seuillage d'Otsu
    try:
        processeur.test_image(image_path)
    except Exception as e:
        logging.error(f"Erreur lors du test de l'image avec la méthode de seuillage d'Otsu : {e}")

if __name__ == "__main__":
    main()



2024-11-27 15:52:31,227 - INFO - Initialisation de ImageProcessor avec l'image : ../coin-counting/dataset/coins_images/coins_images/all_coins/0c0f693a3e.jpg
2024-11-27 15:52:31,248 - INFO - Chargement de l'image depuis le chemin : ../coin-counting/dataset/coins_images/coins_images/all_coins/0c0f693a3e.jpg
2024-11-27 15:52:31,252 - INFO - Calcul des statistiques de l'image
2024-11-27 15:52:31,254 - INFO - Détermination des besoins de prétraitement en fonction des statistiques de l'image
2024-11-27 15:52:31,255 - INFO - Affichage de l'image originale
2024-11-27 15:52:31,256 - INFO - Affichage de l'image
2024-11-27 15:52:31,292 - INFO - Traitement de l'image en fonction des besoins déterminés
2024-11-27 15:52:31,294 - INFO - Calcul des statistiques de l'image
2024-11-27 15:52:31,298 - INFO - Détermination des besoins de prétraitement en fonction des statistiques de l'image
2024-11-27 15:52:31,299 - INFO - Application de l'amélioration du contraste
2024-11-27 15:52:31,301 - INFO - Améliora